# Tarea 2 Maximiliano Ojeda - 201773576-7

## Funciones

In [359]:
from Bio.Data.CodonTable import unambiguous_dna_by_id
from Bio.Seq import Seq


def altcodons(codon, table):
    """List codons that code for the same aminonacid / are also stop.

    @param codon
    @table code table id
    @return list of codons

    """
    tab = unambiguous_dna_by_id[table]

    if codon in tab.stop_codons:
        return tab.stop_codons

    try:
        aa = tab.forward_table[codon]
    except:
        return []

    return [
        k
        for (k, v) in tab.forward_table.items()
        if v == aa and k[0] == codon[0] and k[1] == codon[1]
    ]


def degeneration(codon, table):
    """Determine how many codons code for the same amino acid / are also stop

    @param codon the codon
    @param table code table id
    @param the number of codons also coding for the amino acid codon codes for

    """
    return len(altcodons(codon, table))


def is_x_degenerated(x, codon, table):
    """Determine if codon is x-fold degenerated.

    @param codon the codon
    @param table code table id
    @param true if x <= the degeneration of the codon

    """
    return x <= len(altcodons(codon, table))


def degenerated_subseq(seq, x, table):
    """Get a subsequence consisting of the x-fold degenerated codons only."""
    data = ""
    for i in range(0, len(seq), 3):
        codon = seq[i : i + 3].tostring()
        if isXdegenerated(x, codon, table):
            data += codon
    return data

In [370]:
test_seq = Seq('ACGTTCGAAGGACATCATACCAAAGTC')

## Pregunta 3

### a)

In [361]:
import random
def randomCodons(seq, table=1):
    i = 0
    new_seq = ''
    while i < len(seq):
        cod = seq[i:i+3]
        alt_codons = altcodons(cod, 1)
        new_codon = alt_codons[random.randint(0, len(alt_codons)-1)]
        new_seq += new_codon
        i += 3
    return Seq(new_seq)

In [371]:
randomCodons(test_seq)

Seq('ACTTTCGAAGGTCATCACACGAAGGTG')

### b)

In [372]:
from Bio.SeqUtils import GC
def maxSeq(seq, table=1):
    flag = False
    seqCG = seq
    mayorCG = 0
    for i in range(0,1000):
        new_seq = randomCodons(seqCG)
        valCG = GC(new_seq)
        if valCG > mayorCG:
            mayorCG = valCG
            seqCG = new_seq
    return seqCG

In [373]:
maxSeq(test_seq)

Seq('ACGTTCGAGGGGCACCACACCAAGGTC')

### c)

In [376]:
from Bio import SeqIO
record = SeqIO.read("cds.fna", "fasta")
rand_seq = randomCodons(record.seq)
max_seq = maxSeq(record.seq)

record2 = SeqRecord(
        Seq(rand_seq),
        id=record.id,
        name=record.name,
        description=record.description,
    )
    
with open("cds_random.fna", "w") as output_handle:
    SeqIO.write(record2, output_handle, "fasta")
        
record3 = SeqRecord(
        Seq(max_seq),
        id=record.id,
        name=record.name,
        description=record.description,
    )
    
with open("cds_maxGC.fna", "w") as output_handle:
    SeqIO.write(record3, output_handle, "fasta")

### d)

Se podría utilizar la función GC de Bio.SeqUtils que entrega el porcentaje de G y C para una secuencia en particular.
Entregando como parámetro el porcentaje buscado y una secuencia, luego se itera utilizando GC sobre la secuencia, una vez pasada
por randomCodons() para obtener su valor. Si el valor GC se encuentra dentro de un rango cercano al 
porcentaje deseado se retorna la secuencia que entrego ese porcentaje

## Pregunta 4

In [283]:
from Bio import SeqIO
from Bio.SeqUtils import GC

record = SeqIO.read("NC_005816.fna", "fasta")
table = 1
min_pro_len = 100

In [284]:
ls_orf = []

for strand, nuc in [(+1, record.seq), (-1, record.seq.reverse_complement())]:
    for frame in range(3):
        length = 3 * ((len(record)-frame) // 3) #Multiple of three
        for pro in nuc[frame:frame+length].translate(table).split("*"):
            if len(pro) >= min_pro_len:
                #print("%s...%s - length %i, strand %i, frame %i" % (pro[:30], pro[-3:], len(pro), strand, frame))
                ls_orf.append((GC(pro), pro))

ls_orf.sort(reverse=True)
print("ORFs con mayor porcentaje G+C:")
for i in range(3):
    print(ls_orf[i])
    
print("ORFs con menor porcentaje G+C:")
for i in range(3):
    print(ls_orf[-(i+1)])

ORFs con mayor porcentaje G+C:
(32.773109243697476, Seq('RALTGLSAPGIRSQTSCDRLRELRYVPVSLSASKRAWMLFWSSSASSCSRSHMA...PLQ'))
(20.317460317460316, Seq('GCLMKKSSIVATIITILSGSANAASSQLIPNISPDSFTVAASTGMLSGKSHEML...YRF'))
(18.42105263157895, Seq('QGSGYAFPHASILSGIAMSHFYFLVLHAVKQGFIFGDAKVRTRHINHRFSPYHT...CSD'))
ORFs con menor porcentaje G+C:
(10.4, Seq('WGKLQVIGLSMWMVLFSQRFDDWLNEQEDALQEKVLADLKKLQVYGPELPRPYA...ESK'))
(11.830985915492958, Seq('NQIQGVICSPDSGEFMVTFETVMEIKILHKQGMSSRAIARELGISRNTVKRYLQ...GVA'))
(12.280701754385966, Seq('KSGELRQTPPASSTLHLRLILQRSGVMMELQHQRLMALAGQLQLESLISAAPAL...NPE'))


## Pregunta 5

### a)

In [357]:
import random
from Bio.SeqRecord import SeqRecord

def reorderNucleotidos(file):
    record = SeqIO.read(file, "fasta")
    print(record)
    seq = [i for i in record.seq]
    random.shuffle(seq)
    seq_shuffle = ''.join(seq)
    
    record2 = SeqRecord(
        Seq(seq_shuffle),
        id=record.id,
        name=record.name,
        description=record.description + " shuffle",
    )
    
    with open(file[:-4] + "_shuffle"+ ".fna", "w") as output_handle:
        SeqIO.write(record2, output_handle, "fasta")

### b)

In [358]:
reorderNucleotidos('plasmido.fna')

ID: gi|10955253|ref|NC_002119.1|
Name: gi|10955253|ref|NC_002119.1|
Description: gi|10955253|ref|NC_002119.1| Escherichia coli plasmid CloDF13, complete sequence
Number of features: 0
Seq('AACGTAAAATGTTCAGCGAAAAACCGACATGGTTCACCTATCCTGATAATTGAT...GTT')


## c)

Genoma original
<img src="genoma_original.png">

Genoma aleatorizado
<img src="aleatorio.png">

Evidentemente el genoma original tiene más secuencias encontradas, una diferencia que hay es el largo. En la forma aleatorizada, las secuencias tienen un largo de 200 aproximadamente, mientras que en la original hay de largo sobre 500.